# HAI Deepfake Detection - Colab Training Pipeline

**4가지 핵심 임무:**
1. **코드 동기화 (Git Sync)** - 로컬에서 push한 코드를 Colab에 자동 동기화
2. **자동 환경 세팅 (Environment Setup)** - 필요한 라이브러리 자동 설치
3. **데이터 고속도로 (Data Loading)** - Kaggle API로 대용량 데이터 직접 다운로드
4. **금고 보관 (Model Persistence)** - Google Drive에 학습된 모델 자동 백업

---
## 0. GPU 확인
Colab에서 GPU가 활성화되어 있는지 확인합니다.

In [ ]:
import torch

if torch.cuda.is_available():
    print(f"GPU 사용 가능: {torch.cuda.get_device_name(0)}")
    print(f"CUDA 버전: {torch.version.cuda}")
else:
    print("GPU를 사용할 수 없습니다. 런타임 > 런타임 유형 변" \
    "경 > GPU 선택")

---
## 1. 금고 보관 (Model Persistence) - Google Drive 마운트
학습된 모델을 안전하게 보관하기 위해 Google Drive를 연결합니다.

In [ ]:
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

# 백업 디렉토리 생성
import os
DRIVE_BACKUP_DIR = '/content/drive/MyDrive/HAI_Deepfake'
os.makedirs(f"{DRIVE_BACKUP_DIR}/models", exist_ok=True)
os.makedirs(f"{DRIVE_BACKUP_DIR}/checkpoints", exist_ok=True)
print(f"백업 디렉토리 준비 완료: {DRIVE_BACKUP_DIR}")

---
## 2. 코드 동기화 (Git Sync)
GitHub에서 최신 코드를 가져옵니다. 로컬에서 `git push` 후 이 셀만 실행하면 됩니다.

In [ ]:
import os

# GitHub 설정
REPO_URL = "https://github.com/CBottle/HAI_Deepfake.git"
PROJECT_DIR = "/content/HAI_Deepfake"
BRANCH = "HK"  # <--- 동기화할 브랜치 이름 (HK 또는 main)

if os.path.exists(PROJECT_DIR):
    print(f"기존 프로젝트 발견! {BRANCH} 브랜치 최신 코드로 업데이트 중...")
    %cd {PROJECT_DIR}
    !git fetch --all
    # 브랜치 전환 및 리셋
    !git checkout {BRANCH} || git checkout -b {BRANCH} origin/{BRANCH}
    !git reset --hard origin/{BRANCH}
    print(f"\n{BRANCH} 브랜치 코드 동기화 완료!")
else:
    print(f"프로젝트 클론 중... (브랜치: {BRANCH})")
    !git clone -b {BRANCH} {REPO_URL} {PROJECT_DIR}
    %cd {PROJECT_DIR}
    print(f"\n클론 및 {BRANCH} 브랜치 전환 완료!")

# 현재 커밋 및 브랜치 확인
print("\n현재 정보:")
!git branch --show-current
!git log -1 --oneline
print(f"\n작업 디렉토리: {os.getcwd()}")

---
## 3. 자동 환경 세팅 (Environment Setup)
`requirements.txt`를 읽어 필요한 라이브러리를 자동으로 설치합니다.

In [ ]:
import os

# requirements.txt 경로 확인
req_paths = ['requirements.txt', 'env/requirements.txt']
req_file = None

for path in req_paths:
    if os.path.exists(path):
        req_file = path
        break

if req_file:
    print(f"패키지 설치 중: {req_file}")
    print("="*50)
    !pip install -q -r {req_file}
    print("="*50)
    print("환경 설정 완료!")
else:
    print("requirements.txt를 찾을 수 없습니다.")
    print("기본 패키지를 설치합니다...")
    !pip install -q torch torchvision transformers pillow opencv-python scikit-learn tqdm pyyaml

---
## 4. 데이터 고속도로 (Data Loading) - Kaggle API
Kaggle 서버에서 Colab 서버로 데이터를 직접 다운로드합니다. (내 컴퓨터를 거치지 않아 빠름!)

**사전 준비:**
1. Kaggle 계정에서 `kaggle.json` 다운로드 (Account > API > Create New Token)
2. Google Drive의 `/MyDrive/Kaggle/kaggle.json`에 업로드

In [ ]:
"""
import os

# Kaggle API 설정
KAGGLE_CONFIG_DIR = '/root/.kaggle'
DRIVE_KAGGLE_KEY = '/content/drive/MyDrive/Kaggle/kaggle.json'

os.makedirs(KAGGLE_CONFIG_DIR, exist_ok=True)

if os.path.exists(DRIVE_KAGGLE_KEY):
    !cp {DRIVE_KAGGLE_KEY} {KAGGLE_CONFIG_DIR}/
    !chmod 600 {KAGGLE_CONFIG_DIR}/kaggle.json
    print("Kaggle API 키 설정 완료!")
    KAGGLE_READY = True
else:
    print("kaggle.json을 찾을 수 없습니다.")
    print(f"Google Drive에 파일을 업로드해주세요: {DRIVE_KAGGLE_KEY}")
    KAGGLE_READY = False
"""

In [ ]:
"""
# ============================================
# Kaggle 데이터셋 다운로드
# 아래 DATASET_NAME을 본인의 데이터셋으로 변경하세요!
# ============================================

# 예시 데이터셋 (변경 필요)
DATASET_NAME = "manjilkarki/deepfake-and-real-images"  # Kaggle 데이터셋 경로
DATA_DIR = "./train_data"

if KAGGLE_READY:
    # 기존 데이터가 있는지 확인
    if os.path.exists(DATA_DIR) and len(os.listdir(DATA_DIR)) > 0:
        print(f"데이터가 이미 존재합니다: {DATA_DIR}")
        !ls -la {DATA_DIR}
    else:
        print(f"데이터셋 다운로드 중: {DATASET_NAME}")
        !kaggle datasets download -d {DATASET_NAME} -p {DATA_DIR} --unzip
        print("다운로드 완료!")
else:
    print("Kaggle API가 설정되지 않았습니다.")
    print("더미 데이터를 생성합니다...")
    !python create_dummy_data.py
"""

다운로드X 구글 드라이브에서 zip폴더 가지고 오기

In [ ]:
!cp /content/drive/MyDrive/HAI_Deepfake/extracted_frames.zip /content/extracted_data.zip
!unzip -q /content/extracted_data.zip -d /content/train_data

---
## 5. 학습 실행
GPU를 사용하여 모델을 학습합니다.

In [ ]:
# 학습 시작 전 데이터 확인
import os

print("=" * 50)
print("학습 데이터 구조:")
print("=" * 50)

if os.path.exists('train_data'):
    !ls -la train_data/
    if os.path.exists('train_data/Train'):
        real_count1 = len(os.listdir('train_data/Train/Real'))
        print(f"\ntrain-Real 이미지: {real_count1}개")
        fake_count1 = len(os.listdir('train_data/Train/Fake'))
        print(f"\ntrain-Fake 이미지: {fake_count1}개")
    if os.path.exists('train_data/Validation'):
        real_count2 = len(os.listdir('train_data/Validation/Real'))
        print(f"\nval-Real 이미지: {real_count2}개")
        fake_count2 = len(os.listdir('train_data/Validation/Fake'))
        print(f"\nval-Fake 이미지: {fake_count2}개")
else:
    print("train_data 폴더가 없습니다!")

In [ ]:
# 학습 실행
!python train.py --config config/config.yaml

---
## 7. (선택) 추론 테스트
학습된 모델로 테스트 데이터 추론을 실행합니다.

In [ ]:
!cp /content/drive/MyDrive/HAI_Deepfake/open.zip /content/test_data.zip
!unzip -q /content/test_data.zip -d /content/test_data

In [ ]:
# 추론 실행 (체크포인트 경로 확인 필요)
!python inference.py \
    --model /content/drive/MyDrive/HAI_Deepfake/checkpoints/best_model.pt \
    --test_dir /content/test_data/test_data \
    --output submissions/submission.csv

---
## 빠른 실행 가이드

1. **런타임 > 런타임 유형 변경 > GPU 선택**
2. **런타임 > 모두 실행** 또는 `Ctrl+F9`
3. Google Drive 마운트 승인
4. 학습 완료 후 모델은 자동으로 Drive에 백업됨

### 코드 업데이트 시
로컬에서 `git push` 후, **섹션 2 (코드 동기화)** 셀만 다시 실행하면 됩니다!